# My own exam solutions

In [ ]:
WITH Needed AS(
    SELECT CID 
    FROM INTERNSHIP I1
    WHERE StartDate >= 01/01/2020 AND EndDate <= 31/12/2024 AND EndDate IS NOT NULL
    GROUP BY CID 
    HAVING SUM((EndDate-StartDate)*DailyWage) >
                                            (SELECT SUM((EndDate-StarDate)*DailyWage) 
                                            FROM INTERNSHIP I2
                                            WHERE StartDate >= 01/01/2016 AND EndDate <= 31/12/2019 AND EndDate IS NOT NULL 
                                                  AND I2.CID = I.CID
                                            GROUP BY CID)
)
SELECT Description, Sector, Budget 
FROM PROJECT P, INTERNSHIP I 
WHERE P.ProjNo = I.ProjNo AND P.CID = I.CID AND CID IN Needed AND ((StartDate >= 01/01/2020 AND EndDate <= 31/12/2024) OR 
        (StartDate >= 01/01/2016 AND EndDate <= 31/12/2019)) AND EndDate IS NOT NULL
GROUP BY P.ProjNo, P.CID, Description, Sector, Budget

In [ ]:
WITH Needed AS(
    SELECT CID 
    FROM Internship
    WHERE EndDate IS NULL
    GROUP BY CID 
    HAVING COUNT(*) >9
)
SELECT Description, COUNT(*), AVG(Age), COUNT(DISTINCT Nationality)
FROM PROJECT P, INTERNSHIP I, STUDENT S 
WHERE P.ProjNo = I.ProjNo AND I.CID = P.CID AND S.TaxID = I.TaxID AND P.Budget < 5000 AND CID IN Needed 
GROUP BY P.ProjNo, P.CID, Description 

In [ ]:
WITH Needed AS(
    SELECT CID 
    FROM INTERNSHIP I
    WHERE I.StarDate > 01/01/2022 AND I.EndDate < 31/12/2024 AND I.EndDate IS NOT NULL
    AND I.EndDate IS NOT NULL
    GROUP BY CID 
    HAVING AVG(I.EndDate - I.StartDate) > ( SELECT AVG(I.EndDate - I.StartDate)
                                            FROM INTERNSHIP I2
                                            WHERE I.CID = I2.CID AND I.StartDate >= 01/01/2018 AND I.EndDate <= 31/12/2020 AND I.EndDate IS NOT NULL   ---- careful with date >=
                                            GROUP BY I2.CID)
)
SELECT Description, Sector, Budget
FROM PROJECT P, INTERNSHIP I 
WHERE P.ProjNo = I.ProjNo AND P.CID = I.CID AND I.StartDate > 01/01/2018 AND I.EndDate < 31/12/2020 AND I.StarDate > 01/01/2022 AND I.EndDate < 31/12/2024
    AND I.EndDate IS NOT NULL AND P.CID IN Needed
GROUP BY P.ProjectNo, P.CID, Description, Sector, Budget;

In [ ]:
WITH Needed AS (
    SELECT I.CID 
    FROM STUDENT S, INTERNSHIP I
    WHERE I.TaxID = S.TaxID AND S.Nationality <> 'Italian'
    GROUP BY I.CID 
    HAVING COUNT(DISTINCT ProjNo) > 9
)
SELECT P.Description, COUNT(*) AS NoActive,SUM(EndDate - StartDate), MAX(Age), MIN(Age), COUNT(DISTINCT Nationality)
FROM INTERNSHIP I, STUDENT S, PROJECT P
WHERE EndDate IS NOT NULL AND I.CID IN Needed AND I.TaxID = S.TaxID AND P.ProjNo = I.ProjNo AND P.CID = I.CID            
GROUP BY P.ProjNo, P.CID, P.Description  
        -------- Whatever in SELECT must appear in GROUP BY eventhough you don't need to group by it
        ---- Careful with dates

In [ ]:
WITH Needed AS(
    SELECT CID 
    FROM COMPANY C, INTERNSHIP I
    WHERE C.CID = I.CID AND I.StartDate > 01/01/2023 AND I.EndDate < 31/12/2023
)
SELECT C.City, COUNT(*), AVG(DailWage) --------------------   COUNT(*) counts all
FROM COMPANY C, INTERNSHIP I
WHERE C.CID = I.CID AND C.CID NOT IN Needed
GROUP BY C.City;

In [ ]:
CREATE TRIGGER MANAGE_Booking
AFTER INSERT ON BOOKING
FOR EACH ROW 
DECLARE                                          ------------ DECLARE not DEFINE
plateNum VARCHAR(15);
cost NUMBER;                                          ----------------not INT  but NUMBER
BEGIN
WITH Available AS(
    SELECT LicensePlate, CostPerDay
    FROM VEHICLE V, VEHICLE_AVAILABILITY VA 
    WHERE V.LicensePlate = VA.LicensePlate AND :NEW.Category = V.Category AND :NEW.NoSeat = V.NoSeats AND Date>=:NEW.StartDate AND Date<=:NEW.EndDate
            AND Status = 1
    GROUP BY LicensePlate, CostPerDay
    HAVING COUNT(*)= :NEW.StartDate - :NEW.EndDate+1
)
SELECT LicensePlate, CostPerDay INTO plateNum, cost
FROM Available A1
WHERE CostPerDay = (SELECT MIN(CostPerDay)
                    FROM Available A2)
IF (plateNum IS NULL) THEN                          --------------------  IF (______) THEN
RAISE_APPLICATION_ERROR(-20500, 'not found');
ELSE
UPDATE VEHICLE_AVAILABILITY
SET Status = 0
WHERE Date>= :NEW.StarDate AND Date <= :NEW.EndDate AND LicensePlate = plateNum;

UPDATE VEHICLE
SET NumRentals = NumRentals + 1
WHERE LicensePlate = plateNum;                     ---------------------  after every complete action put    ;

INSERT INTO NOTIFICATION
VALUES(:NEW.TaxID, :NEW.StartDate, plateNum, (:New.EndDate - :NEW.StartDate+1)*cost);
END IF;
END;
   --------------- child attributes are merged as optional


## 24/05/27

In [ ]:
WITH Needed AS(
    SELECT WID 
    FROM VEHICLE V, INSPECTION I 
    WHERE V.FuellingType = 'LPG-fueled' AND V.RegistrationYear <2015 AND 
        AND V.LicensePlate# = I.LicensePlate#; 
)
SELECT Name, Address 
FROM WORKSHOP W 
WHERE W.CIty = 'Turin' AND W.WID NOT IN Needed;

In [ ]:
WITH Needed AS(
    SELECT SSN 
    FROM VEHICLE V1
    WHERE RegistrationYear > 01/01/2010
    GROUP BY SSN 
    HAVING COUNT(*)>=2;
)
SELECT C.LastName,V.LicensePlate#, V.Maker, COUNT(*), AVG(I.Cost), COUNT(DISTINCT I.WID)
FROM CUSTOMER C, VEHICLE V, INSPECTION I 
WHERE C.SSN = V.SSN AND I.LicensePlate# = V.LicensePlate# AND BirthDate >01/01/1980 AND V.SSN IN Needed
GROUP BY C.SSN, C.LastName, v.LicensePlate#, V.Maker;

In [ ]:
WITH Needed AS(
    SELECT I.WID, W.City, COUNT(*) AS NoInspections , COUNT(DISTINCT Model) AS NoModels, SUM(Cost) AS Total
    FROM INSPECTION I, VEHICLE V, CUSTOMER C, WORKSHOP W 
    WHERE C.BirthDate >= 01/01/1970 AND C.BirhtDate <01/01/1981 AND
        I.LicensePlate# = V.LicensePlate# AND C.SSN = V.SSN 
        W.WID = I.WID
    GROUP BY I.WID, W.City 
    HAVING COUNT(DISTINCT I.LicensePlate#) >=200;
)
SELECT Name, Address, NoInspections, NoModels, Total
FROM WORKSHOP W, Needed N1
WHERE W.WID = N1.WID AND N1.NoInspections =(SELECT MAX(NoInspections)
                                            FROM Needed N2
                                            WHERE N1.City = N2.City)

## 2nd simulation

In [ ]:
WITH NeededSize AS(
    SELECT TeacherID 
    FROM MATERIAL_UPLOADED_BY_TEACHER M 
    WHERE Size < 10;
)
NeededType AS(
    SELECT TeacherID 
    FROM MATERIAL_UPLOADED_BY_TEACHER M 
    WHERE Category <> 'Exercises'
)                                     ----------- you can use OR in WHERE too
SELECT Name, Surname 
FROM TEACHER T  
WHERE T.Course = 'Analysis I'AND T.TeacherID NOT IN Needed AND T.TeacherID NOT IN NeededType;

In [ ]:
WITH Needed AS(
    SELECT TeacherID 
    FROM MATERIAL_UPLOADED_BY_TEACHER M, VISUALIZATIONS V 
    WHERE M.MatID = V.MatID AND M.Format = 'ZIP'
    GROUP BY MatID, TeacherID 
    HAVING SUM(VisualizationsNum)> 100;
)
SELECT Course, Date, COUNT(DISTINCT TeacherID), SUM(Minutes) 
FROM TEACHER T, TEACHER_CONNECTIONS TC 
WHERE T.TeacherID = TC.TeacherID AND T.TeacherID IN Needed 
GROUP BY Course, Date;

In [1]:
WITH Needed AS(
    SELECT TeacherID 
    FROM TEACHER T, TEACHER_CONNECTIONS TC 
    WHERE T.TeacherID = TC.TeacherID
    GROUP BY TeacherID, Course 
    HAVING AVG(Minutes) > (SELECT AVG(Minutes)
                          FROM TEACHER_CONNECTIONS TC2, TEACHER T2 
                          WHERE T2.TeacherID = TC2.TeacherID AND
                                T.Course = T2.Course);
)
SELECT Category, COUNT(DISTINCT Format), MAX(Size)
FROM DOCUMENTS_UPLOADED_BY_TEACHER D 
WHERE TeacherID IN Needed AND NumPages > (SELECT AVG(NumPages)
                                          FROM DOCUMENTS_UPLOADED_BY_TEACHER D2 
                                          WHERE D2.Category = D.Category)
GROUP BY Category;

SyntaxError: invalid syntax (3971460451.py, line 1)

## Simulation 1

In [ ]:
WITH Needed AS(
    SELECT CodM 
    FROM PLAYER P, YELLOW_CARDS Y
    WHERE P.CodP = Y.CodP AND P.Role = 'Striker' 
    GROUP BY CodM
    HAVING COUNT(*) > 3;
)
SELECT CodM, Date, COUNT(*)
FROM MATCH M, GOAL G 
WHERE M.CodM = G.CodM AND CodM NOT IN Needed
GROUP BY CodM, Date, CodP;

In [ ]:
SELECT Name, Surname, Date 
FROM PLAYER P, MATCH M, GOAL G 
WHERE P.CodP = G.CodP AND M.CodM = G.CodM AND Nation = 'Paraguay'
    AND Date = (SELECT MIN(Date)
                FROM GOAL G2, MATCH M2
                WHERE G2.CodM = M2.CodM AND G2.CodP = G.CodP);

SELECT Name, Surname, MIN(Date) 
FROM PLAYER P, MATCH M, GOAL G 
WHERE P.CodP = G.CodP AND M.CodM = G.CodM AND Nation = 'Paraguay'
GROUP BY CodP, Name, Surname;


In [ ]:
SELECT Name, Surname, Nation 
FROM PLAYER P, GOAL G 
WHERE G.CodP = P.CodP AND MinutesIntoTheMatch > 10
GROUP BY P.CodP, Name, Surname, Nation 
HAVING 5*COUNT(DISTINCT CodM)>= 
                  (SELECT COUNT(DISTINCT CodM)
                   FROM GOAL G2, MATCH M
                   WHERE G2.CodP = G.CodP
                        G2.CodM = M.CodM AND M.Date >= 01/01/2020 AND M.Date < 01/01/2021)

## Simulation 2

In [ ]:
SELECT Name, SUM(Quantity) 
FROM TOY_BUILDING_SET T, ORDER_INLCUDE OI
WHERE Category = 'Marvel'AND NumPieces >300 AND OI.SID = T.SID 
GROUP BY SID, Name 
HAVING COUNT(*) >= 50

In [ ]:
WITH Needed AS(
    SELECT TaxID
    FROM ORDER O, ORDER_INCLUDE OI, TOY_BUILDING_SET T
    WHERE O.OID = OI.OID AND T.SID = OI.SID AND Date >= 01/01/2022 
            AND Date <= 31/12/2022
    GROUP BY TaxID, O.OID 
    HAVING COUNT(DISTINCT Category) >3;
)
SELECT City, Date, COUNT(DISTINCT TaxID)
FROM CUSTOMER C, ORDER O 
WHERE O.TaxID = C.TaxID AND O.TaxID NOT IN Needed AND 
        Date >= 01/01/2022 AND Date <= 31/12/2022 
GROUP BY City, Date;

In [ ]:
WITH Needed AS(
    SELECT SID
    FROM ORDER_INCLUDE OI1, ORDER O1 
    WHERE O1.OID = OI1.OID AND Date>= 01/05/2023 AND Date <= 31/05/2023 
    GROUP BY SID 
    HAVING SUM(Quantity*UnitCost) >= (SELECT 2*SUM(Quantity*UnitCost)
                                      FROM ORDER_INCLUDE OI2, ORDER O2 
                                      WHERE O1.OID = OI1.OID AND 
                                         Date>= 01/04/2023 AND Date <= 31/04/2023
                                         AND  O2.SID = O1.SID)
)
SELECT Category, COUNT(DISTINCT SID) 
FROM TOY_BUILDING_SET T, ORDER_INCLUDE OI, ORDER O 
WHERE T.SID = OI.SID AND O.OID = OI.OID AND 
        SID IN Needed AND Date>= 01/05/2023 AND Date <= 31/05/2023 
GROUP BY Category;

## Simulation 3

In [ ]:
WITH Needed AS(
    SELECT CID, Title, MAX(SizeKB) AS MaxAudio
    FROM MESSAGE M1, CHAT C1, USER U 
    WHERE M1.CID = C1.CID AND U.UID = C1.UIAdmin AND 
        C1.CreationDate >= 01/01/2023 AND C1.CreationDate < 01/01/2024 
        AND U.BirthDate < 01/01/1990 AND M.Type = 'Audio' 
    GROUP BY CID, Title;
)
SELECT Title, MaxAudio
FROM Needed
WHERE CID NOT IN (SELECT CID
                  FROM CHAT C, MESSAGE M 
                  WHERE C.CID = M.CID AND Type = 'Image');

In [ ]:
WITH Needed AS(
    SELECT UID
    FROM USER U, CHAT C 
    WHERE U.UID = C.UIAdmin AND City = 'Milan' AND 
        CreationDate < 01/01/2023 AND CreationDate >= 01/01/2022
    GROUP BY UID 
    HAVING COUNT(*)>=5;
)
SELECT Type, COUNT(*), SUM(SizeKB)
FROM MESSAGE M
WHERE  UIDSender IN Needed
GROUP BY Type
HAVING COUNT(DISTINCT UIDSender)>=10;

In [ ]:
WITH Needed AS(
    SELECT SubmitDate AS Date, CID, COUNT(*) AS NoMessages 
    FROM MESSAGE
    GROUP BY SubmitDate, CID;
)
SELECT SubmitDate, Name, Surname, Title 
FROM USER U, CHAT C, MESSAGE M 
WHERE U.UID = C.UIAdmin AND M.CID = C.CID 
GROUP BY M.SubmitDate, CID, Name, Surname, Title
HAVING COUNT(*) >= (SELECT AVG(NoMessages)
                    FROM Needed
                    WHERE Needed.Date = M.SubmitDate)

## Simulation 4

In [ ]:
WITH Need AS(
    SELECT SID 
    FROM SONG S, RADIO_PROGRAMMING P 
    WHERE S.Language = 'Italian' AND S.SID = P.SID 
        AND StartTime <= '14:00' 
)
SELECT Name, Network
FROM WEBRADIO W, RADIO_PROGRAMMING RP, SONG S1
WHERE RP.WID = W.WID AND S1.SID = RP.SID 
    S1.Language = 'Italian' AND SID NOT IN (SELECT SID FROM Needed)
GROUP BY WID, Name, Network
HAVING COUNT(DISTINCT SID) > 100

In [ ]:
WITH Needed AS(
    SELECT SID 
    FROM AUDIENCE A 
    WHERE Date >= 01/01/2020 AND Date < 01/01/2021
    GROUP BY SID 
    HAVING COUNT(NoAudience) > 90000;
)
SELECT Title, COUNT(DISTINCT WID), MAX(Date)
FROM SONG S, RADIO_PROGRAMMING P 
WHERE S.SID IN Needed AND S.SID = P.SID 
GROUP BY P.SID, Title;

In [ ]:
WITH Needed AS(  
    SELECT Province, City, SUM(NoAudience) CityNo
    FROM AUDIENCE A, SONG S, Location L1
    WHERE Date >= 01/01/2020 AND Date < 01/01/2021 AND S.MusicGenre = 'R&B'
        AND S.SID = A.SID AND L1.City = A.City
    GROUP BY Province, City;
)
SELECT Province, City 
FROM AUDIENCE A1, SONG S1, Location L
WHERE Date >= 01/01/2020 AND Date < 01/01/2021 AND S1.MusicGenre = 'R&B'
        AND S1.SID = A1.SID AND A1.City = L.City
GROUP BY Province,City
HAVING SUM(NoAudience) >= (SELECT AVG(CityNo)
                           FROM Needed N
                           WHERE N.Province = L.Province);

In [ ]:

COURSE(CID, CourseName, Type, MaxNoStudents*)
COURSE_TOPICS(CID, TopicName)
COURSE_EDITION_OF_AVAILABLE(CID, StartDate, Duration, EndDate, PID)
ONLINE_PLATFORM(PID, Platform_Name)
STUDENT(ID, Name, Surname, Facebook*)
UNIVERSITY_RESIDENCE(RID, Phone, Address)
LIVES_IN(ID, RID, StartDate, EndDate*)
ENROLLED(ID, RID, StartDate)

ENROLLED(CID, StartDate) REFERENCES COURSE_EDITION_OF_AVAILABLE(CID, StartDate)
LIVES_IN(RID) REFERENCES UNIVERSITY_RESIDENCE(RID)

In [ ]:
-- consider the case of not present in license_purchased and notification

CREATE TRIGGER Manage_purchases
AFTER INSERT ON SOFTWARE_PURCHASE_REQUEST
FOR EACH ROW 
DECLARE 
varComp VARCHAR(15);
varSoft VARCHAR(15);
cost NUMBER;
noPurchased NUMBER;
BEGIN
SELECT IdC, IdSW INTO varComp, varSoft
FROM LICENSES_PURCHASED
WHERE IdC = :NEW.IdC AND IdSW = :NEW.IdSW;

SELECT CostPerLicense INTO cost 
FROM SOFTWARE 
WHERE :NEW.IdSW = IdSW;

IF (varComp IS NULL and varSoft IS NULL)THEN --- not yet entered

INSERT INTO LICENSES_PURCHASED 
VALUES(varComp, varSoft, :NEW.NoRequestedLicenses);

INSERT INTO SOFTWARE_PURCHASE_NOTIFICATION 
VALUES(:NEW.IdR,varComp, varSoft, :NEW.NoRequestedLicenses, 
        cost*(:NEW.NoRequestedLicenses), :NEW.NoRequestedLicenses);

ELSE  --- exists entry
SELECT MAX(NoPurchasedLicenses) INTO noPurchased
FROM SOFTWARE_PURCHASE_NOTIFICATION
WHERE IdC = :NEW.IdC AND IdSW = :NEW.IdSW;

UPDATE LICENSE_PURCHASED
SET NoPurchasedLicenses = NoPurchasedLicenses + :NEW.NoRequestedLicenses
WHERE IdC = :NEW.IdC AND IdSW = :NEW.IdSW;
INSERT INTO SOFTWARE_PURCHASE_NOTIFICATION 
VALUES(:NEW.IdR,varComp, varSoft, :NEW.NoRequestedLicenses, 
        cost*(:NEW.NoRequestedLicenses), noPurchased + :NEW.NoRequestedLicenses);
END IF;
END;